# Notebook to plot snowlines and respective images and select the annual ELA (maximum elevation snowline)

In [ ]:
# Install packages not already in environment
# !pip install geopandas rioxarray geedim

In [ ]:
# Import packages
import os
import matplotlib.pyplot as plt
import glob
import pandas as pd
import geopandas as gpd
import numpy as np
import sys
from shapely import wkt
import sys
import json
import sys
import ee
import datetime
import ast

In [ ]:
# Authenticate your Google Earth Engine account
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# If using Google Colab, mount Google Drive so you can access the files in this folder
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

In [ ]:
# Uncomment the line below to locate any needed paths in directory
# os.listdir('drive/MyDrive/Research/PhD/snow_cover_mapping/snow_cover_mapping_application/study-sites/')

In [ ]:
##### MODIFY HERE #####
# specify site name
site_name = 'RGI60-01.00037'
# path to snow-cover-mapping-application/
# RAINEY'S PATH:
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping-application/'
# ALEXANDRA'S PATH:
# base_path = ('drive/MyDrive/snow_cover_mapping_application/snow-cover-mapping-application/')
# path to study site
study_site_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites/' + site_name
# path where figures will be saved
figures_out_path = study_site_path + 'figures/'
# path where manual ELA estimates will be saved
out_path = os.path.join(study_site_path, 'imagery', 'ELAs')
# define month range for filtering snowlines from file (includes both months)
month_start = 7
month_end = 10
#######################

# -----Determine other folders
# path to PlanetScope image mosaics
PS_im_path = os.path.join(study_site_path, 'imagery', 'PlanetScope', 'mosaics')
# path to snowline files
sl_est_path = os.path.join(study_site_path, 'imagery', 'snowlines')

# -----Load dataset characteristics
dataset_dict_fn = os.path.join(base_path, 'inputs-outputs', 'datasets_characteristics.json')
dataset_dict = json.load(open(dataset_dict_fn))

# -----Load functions from file
sys.path.append(os.path.join(base_path, 'functions'))
import manually_select_ELAs_utils as f



In [ ]:
# -----Load and plot glacier outline shapefile to see if the geometry is wacky!
RGI_outline = os.path.join(study_site_path, 'AOIs', '*.shp') # file name
shapefiles = glob.glob(RGI_outline) # check if file name in directory

if not shapefiles:
    print(f"No shapefiles found in {RGI_outline}")
else:
    AOI_fn = shapefiles[0]
    AOI = gpd.read_file(AOI_fn)
    AOI.plot()
    plt.title('Glacier Outline Plot: ' + site_name)
    plt.show()

In [ ]:
# -----Load and compile snowline files
# Define year for selecting the ELA
year = '2022'

# grab snowline file names
sl_est_fns = glob.glob(os.path.join(sl_est_path, '*snowline.csv'))
# only include snowlines starting in 2016 (time series too sparse before then)
sl_est_fns = [sl_est_fn for sl_est_fn in sl_est_fns if year in sl_est_fn]

# compile all snowline files into one DataFrame
sl_est_full = gpd.GeoDataFrame()
for fn in sl_est_fns:
    # read file
    sl_est_df = pd.read_csv(fn)
    # create a geopandas GeoDataFrame from the DataFrame
    sl_est_df.loc[sl_est_df['geometry']=='[]', 'geometry'] = None
    sl_est_df['geometry'] = sl_est_df['geometry'].apply(wkt.loads)
    sl_est_gdf = gpd.GeoDataFrame(sl_est_df, crs='EPSG:4326',
                                  geometry=sl_est_df['geometry'])
    # concatenate to df
    sl_est_full = pd.concat([sl_est_full, sl_est_gdf])
# don't include PlanetScope
sl_est_full = sl_est_full.loc[sl_est_full['dataset'] != 'PlanetScope']
# read dates as datettime64, only include snowlines for July - November
sl_est_full['datetime'] = sl_est_full['datetime'].astype('datetime64[ns]')
sl_est_full['month'] = sl_est_full['datetime'].values.astype('datetime64[M]').astype(int) % 12 + 1
sl_est_full = sl_est_full.loc[(sl_est_full['month'] >= month_start) & (sl_est_full['month'] <= month_end)]
# remove month column
sl_est_full = sl_est_full.drop('month', axis=1)
# remove Unnamed column if present
if 'Unnamed: 0' in list(sl_est_full.columns):
    sl_est_full = sl_est_full.drop('Unnamed: 0', axis=1)
# renumber, sort by date
sl_est_full = sl_est_full.sort_values(by=['datetime']).reset_index(drop=True)

# -----Reproject AOI and snowlines to WGS84
AOI_UTM = AOI.to_crs(sl_est_full['HorizontalCRS'][0])

sl_est_full

In [ ]:
checkboxes = f.manual_snowline_filter_plot(sl_est_full, dataset_dict, AOI_UTM, PS_im_path)

## 2. View resulting annual ELAs

In [ ]:
# -----Grab ELA selections from checkboxes
values = np.array([x.value for x in checkboxes])
IELAs = np.ravel(np.argwhere(values==True))
ELAs = sl_est_full.iloc[IELAs]

# -----Plot
plt.figure(figsize=(12,6))
plt.plot(sl_est_full['datetime'].astype('datetime64[ns]'), sl_est_full['snowline_elevs_median_m'], '.k')
plt.plot(ELAs['datetime'].astype('datetime64[ns]'), ELAs['snowline_elevs_median_m'], '*m', markersize=15)
plt.grid()
plt.ylabel('Median snowline elevation [m]')

## 3. Save annual ELAs to file

When you're satisfied with your ELA selections, save them!

In [ ]:
# define file name and where to save it
out_fn = site_name + '_manually_selected_ELAs_' + year + '.csv'
out_path = study_site_path + 'ELAs/'

# make output directory (out_path) if it doesn't exist
if not os.path.exists(out_path):
  os.mkdir(out_path)
  print('Made directory for outputs: '+out_path)

# save to file
ELAs.to_csv(out_path + out_fn, index=False)
print('ELAs saved to file: ' + out_path + out_fn)

## Optional. Combine CSVs into one

In [ ]:
# Grab all the CSVs in your out path
fns = sorted(glob.glob(out_path + '*.csv'))
year_start = fns[0][-8:-4]
year_end = fns[-1][-8:-4]

# Create a new Data Frame
ELAs = pd.DataFrame()

# Loop through the CSVs
for fn in fns:
    ELA = pd.read_csv(fn)
    ELAs = pd.concat([ELAs, ELA])

# Save to file
out_fn = (out_path + 'RGI60-01.11788' + '_manually_selected_ELAs_'
            + year_start + '_' + year_end + '.csv')
ELAs.to_csv(out_fn, index=False)
print('Compiled ELAs saved to file: ' + out_fn)

In [ ]:
# Delete individual files - CHECK THE NEW FILE BEFORE DELETING THE OLD ONES
for fn in fns:
    os.remove(fn)
    print(fn + ' deleted from file.')